# Table of Contents
* [overview](#overview)


In [6]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
from tqdm import tqdm as tqdm
from collections import defaultdict
from collections import Counter
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2

In [7]:
%%capture
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pylab as plt
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light solarized
#%base16_mplrc dark solarized
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['figure.figsize'] = (16.0, 10.0)

In [3]:
import os
import cv2
import PIL.Image as Image

# overview

<img  src="robot_dataset_section.jpg"/>

# THOR 

In [8]:
import ai2thor.controller
import pickle
import json
controller = ai2thor.controller.Controller()
controller.start()

val1_id = 'FloorPlan_RVal1'

thor-OSXIntel64-00e7bb0602270cc446028485d438011b389e64ff: [ 100%   9.3 MiB/s]  of 1062MB
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@2/2.7.17/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python@2/2.7.17/Frameworks/Python.framework/Versions/2.7/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/Users/schwenk/wrk/transfer/src/ai2thor/ai2thor/controller.py", line 733, in _start_unity_thread
    raise Exception("command: %s exited with %s" % (command, returncode))
Exception: command: ['/Users/schwenk/.ai2thor/releases/thor-OSXIntel64-00e7bb0602270cc446028485d438011b389e64ff/thor-OSXIntel64-00e7bb0602270cc446028485d438011b389e64ff.app/Contents/MacOS/thor-OSXIntel64-00e7bb0602270cc446028485d438011b389e64ff', '-screen-fullscreen', '0', '-screen-quality', '7', '-screen-width', '300', '-screen-

# Volume 

## # scenes

In [29]:
scene_names_df = pd.read_csv('/Users/schwenk/wrk/transfer/src/ai2thor/scene_list.txt', header=None)
scene_names_df.columns = ['name']
scene_names_df['split'] = scene_names_df.name.str.lower().str.split('_').apply(lambda x: ''.join([i for i in x[1] if not i.isdigit()]))

In [36]:
scene_counts = pd.DataFrame(scene_names_df.split.value_counts())
scene_counts

,split
train,75
rtest,10
rval,4


## # assets